In [2]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

x = load_iris().data
y = load_iris().target

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(112, 4)

In [4]:
def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    theta = np.random.rand(X.shape[1], 1)
    print(theta)
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            r = np.random.randint(0, m-1)
            xi, yi = X[r:r+1, :], y[r:r+1]
            diff = xi.T @ ((xi @ theta) - yi)
            eta = learning_schedule(epoch * m + n)
            theta = theta - (diff * eta)
    return theta

theta = SGD(x_train, y_train, 50, 0.1)

print(x_train[0:0, :])

print(theta)

preds = x_test @ theta
preds = np.rint(preds)
print(f'SGD Acc: {accuracy_score(y_test, preds)}')

[[ 0.19117887]
 [ 0.60531646]
 [ 0.56623743]
 [ 0.30867393]]
[]
[[-0.18435486]
 [ 0.0859226 ]
 [ 0.35538651]
 [ 0.41318002]]
SGD Acc: 0.9736842105263158


In [5]:
def normal(X):
    return X / X.max(axis=0)

In [6]:
import numpy as np
def predict(X, theta):
    return np.c_[np.ones(len(X)), normal(X)] @ theta

print(np.random.rand(1, 14))

[[ 0.05600277  0.68066653  0.64753314  0.14061039  0.38958653  0.9920309
   0.2677404   0.46636503  0.73003877  0.4896937   0.95906199  0.5329149
   0.13971411  0.01799687]]


In [7]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = (((X @ theta.T) - y.T).T @ nX)
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = BGD(x_train, y_train, 1000, 0.001)

print(theta)

preds = predict(x_test, theta)
preds = np.rint(preds)
print(f'\nBGD Acc: {accuracy_score(y_test, preds)}')

[[-0.07008974]
 [-0.20559497]
 [-0.33030424]
 [ 1.30353235]
 [ 1.50961471]]

BGD Acc: 0.9473684210526315


In [8]:
def MBGD(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    print(m)
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T
        
theta = MBGD(x_train, y_train, 100, 20, 0.01)

print(theta)

preds = predict(x_test, theta)
preds = np.rint(preds)
print(f'\n MBGD Acc: {accuracy_score(y_test, preds)}')

112
[[-0.25352927]
 [ 0.24262577]
 [-0.41807744]
 [ 1.12199328]
 [ 1.5230221 ]]

 MBGD Acc: 0.9473684210526315


In [9]:
import numpy.linalg as LA

def Ridge(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    print(m)
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - eta * (gradients + (eta * LA.norm(theta)**2))
    return theta.T

theta = Ridge(x_train, y_train, 100, 20, 0.01)

print(theta)

preds = predict(x_test, theta)
preds = np.rint(preds)
print(f'\nBGD Acc: {accuracy_score(y_test, preds)}')

112
[[-0.26034416]
 [-0.07910107]
 [-0.14838383]
 [ 1.07258123]
 [ 1.69487337]]

BGD Acc: 0.9210526315789473


In [26]:
def Logistic(X, y, n_iter, eta, alpha):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T

def proba(y):
    return 1 / (1 + np.exp(-y))

proba(-0.02)

0.4950001666600003